In [1]:
import inro.modeller as m
import os
import glob
mm = m.Modeller()
eb = mm.emmebank
import inro.emme.matrix as _matrix
import pandas as pd
import numpy as np
from copy import deepcopy

matrix_calculator = m.Modeller().tool("inro.emme.matrix_calculation.matrix_calculator")
create_matrix =  m.Modeller().tool("inro.emme.data.matrix.create_matrix")
change_matrix = m.Modeller().tool("inro.emme.data.matrix.change_matrix_properties") 
import_matrices = mm.tool("tmg.input_output.import_binary_matrix")  
matrix_transaction = mm.tool('inro.emme.data.matrix.matrix_transaction')
create_extra = m.Modeller().tool("inro.emme.data.extra_attribute.create_extra_attribute")
network_calc= m.Modeller().tool("inro.emme.network_calculation.network_calculator")

def clear_matrices(emmebank, exception_array = [""], type = "FULL"):
    for matrix in emmebank.matrices():
        if matrix.type == type:
            if matrix.id not in exception_array:
                emmebank.delete_matrix(matrix.id)

def import_matrices_from_directory(matrix_folder, matrix_list, extension, scenario):
    #imports all matrices in directory and subfolders of the specified extension type
    #matrix_list should be in the format:
    #   {file_name1: matrix_num1, file_name2: matrix_num2, ...}
    import os

    for root, dirs, files in os.walk(matrix_folder):
        for matrix_file in files:
            if matrix_file.endswith(extension):
                name = os.path.splitext(matrix_file)[0]
                if name in matrix_list:
                    matrix_number = int(matrix_list[name][2:])
                    import_matrices(4, matrix_number ,os.path.join(root, matrix_file),scenario, name.replace("skim.","").replace("transit.","").replace("peak","pk").replace(" ","_") )
                    print ("Imported matrix " + matrix_list[name])
                    change_matrix(matrix = matrix_list[name],
                        matrix_name = name.replace("skim.","").replace("transit.","").replace("peak","pk").replace(" ","_")[:40],
                        matrix_description = name)
    
    #check
    for name in matrix_list:
        if eb.matrix(matrix_list[name]): 
            n_matrix = eb.matrix(matrix_list[name]).get_numpy_data(scenario)
            if n_matrix.sum() == 0 :
                print ("Matrix %s was not imported, or contains no values!" % name)
        else:
            print ("Matrix %s was not imported!" % name)
    print ("Finished Importing Matrices")

In [2]:
sc_num_am = 7000
sc_num_md = 7001

#auto occupancy
sov_oc = 1.2
hov2_oc = 2.2
hov3_oc = 3.3

sc_am = eb.scenario(sc_num_am)
sc_md = eb.scenario(sc_num_md)
ntwk_am = sc_am.get_network()
ntwk_md = sc_md.get_network()
ntwks = [ntwk_am, ntwk_md]

In [3]:
## zone grouping definitions ## 
zones_pd1 = "gf1"
zones_airports = "ga1"
zones_ugc = "gb4"
zones_inner = "gc1"
zones_outer = "gc2"
zones_university = "gd1"
zones_employment = "ge1"
zones_hospitals = "gh1"
shed_central = "gs10;gs11"
rural_central = "gs11"
shed_east = "gs20;gs21"
rural_east = "gs21"
shed_midwest = "gs30;gs31"
rural_midwest = "gs31"
shed_southwest = "gs40;gs41"
rural_southwest = "gs41"
shed_west = "gs50;gs51"
rural_west = "gs51"
shed_north = "gs60;gs61"
rural_north = "gs61"
shed_niagara = "gs70;gs71"
rural_niagara = "gs71"
shed_urban = "gs10;gs20;gs30;gs40;gs50;gs60;gs70"
shed_rural = "gs11;gs21;gs31;gs41;gs51;gs61;gs71"
urban_lu = "gb2;gb3;gb4"

zones_array = ["all",zones_pd1,shed_central,shed_east,shed_midwest,shed_southwest,shed_west,shed_north,shed_niagara]
rural_array = [shed_rural,zones_pd1,rural_central,rural_east,rural_midwest,rural_southwest,rural_west,rural_north,rural_niagara]
zones_array_2 = [zones_pd1,shed_central,shed_east,shed_midwest,shed_southwest,shed_west,shed_north,shed_niagara]

zones_vkt = [{"all":{"vkt": 0.0, "cong_vkt": 0.0, "truck_vkt": 0.0,"cong_truck_vkt":0.0, "cost":0.0,
                "pht": 0.0, "ff_pht":0.0, "trk_pht":0.0,"ff_trk_pht":0.0,"bridges":0.0},
         zones_pd1:{"vkt": 0.0, "cong_vkt": 0.0, "truck_vkt": 0.0,"cong_truck_vkt":0.0, "cost":0.0,
                "pht": 0.0, "ff_pht":0.0, "trk_pht":0.0,"ff_trk_pht":0.0,"bridges":0.0},
         shed_central:{"vkt": 0.0, "cong_vkt": 0.0, "truck_vkt": 0.0,"cong_truck_vkt":0.0, "cost":0.0,
                "pht": 0.0, "ff_pht":0.0, "trk_pht":0.0,"ff_trk_pht":0.0,"bridges":0.0},
         shed_east:{"vkt": 0.0, "cong_vkt": 0.0, "truck_vkt": 0.0,"cong_truck_vkt":0.0, "cost":0.0,
                "pht": 0.0, "ff_pht":0.0, "trk_pht":0.0,"ff_trk_pht":0.0,"bridges":0.0},
         shed_midwest:{"vkt": 0.0, "cong_vkt": 0.0, "truck_vkt": 0.0,"cong_truck_vkt":0.0, "cost":0.0,
                "pht": 0.0, "ff_pht":0.0, "trk_pht":0.0,"ff_trk_pht":0.0,"bridges":0.0},
         shed_southwest:{"vkt": 0.0, "cong_vkt": 0.0, "truck_vkt": 0.0,"cong_truck_vkt":0.0, "cost":0.0,
                "pht": 0.0, "ff_pht":0.0, "trk_pht":0.0,"ff_trk_pht":0.0,"bridges":0.0},
         shed_west:{"vkt": 0.0, "cong_vkt": 0.0, "truck_vkt": 0.0,"cong_truck_vkt":0.0, "cost":0.0,
                "pht": 0.0, "ff_pht":0.0, "trk_pht":0.0,"ff_trk_pht":0.0,"bridges":0.0},
         shed_north:{"vkt": 0.0, "cong_vkt": 0.0, "truck_vkt": 0.0,"cong_truck_vkt":0.0, "cost":0.0,
                "pht": 0.0, "ff_pht":0.0, "trk_pht":0.0,"ff_trk_pht":0.0,"bridges":0.0},
         shed_niagara:{"vkt": 0.0, "cong_vkt": 0.0, "truck_vkt": 0.0,"cong_truck_vkt":0.0, "cost":0.0,
                "pht": 0.0, "ff_pht":0.0, "trk_pht":0.0,"ff_trk_pht":0.0,"bridges":0.0}},
        {"all":{"vkt": 0.0, "cong_vkt": 0.0, "truck_vkt": 0.0,"cong_truck_vkt":0.0,
                "pht": 0.0, "ff_pht":0.0, "trk_pht":0.0,"ff_trk_pht":0.0},
         zones_pd1:{"vkt": 0.0, "cong_vkt": 0.0, "truck_vkt": 0.0,"cong_truck_vkt":0.0,
                "pht": 0.0, "ff_pht":0.0, "trk_pht":0.0,"ff_trk_pht":0.0},
         shed_central:{"vkt": 0.0, "cong_vkt": 0.0, "truck_vkt": 0.0,"cong_truck_vkt":0.0,
                "pht": 0.0, "ff_pht":0.0, "trk_pht":0.0,"ff_trk_pht":0.0},
         shed_east:{"vkt": 0.0, "cong_vkt": 0.0, "truck_vkt": 0.0,"cong_truck_vkt":0.0,
                "pht": 0.0, "ff_pht":0.0, "trk_pht":0.0,"ff_trk_pht":0.0},
         shed_midwest:{"vkt": 0.0, "cong_vkt": 0.0, "truck_vkt": 0.0,"cong_truck_vkt":0.0,
                "pht": 0.0, "ff_pht":0.0, "trk_pht":0.0,"ff_trk_pht":0.0},
         shed_southwest:{"vkt": 0.0, "cong_vkt": 0.0, "truck_vkt": 0.0,"cong_truck_vkt":0.0,
                "pht": 0.0, "ff_pht":0.0, "trk_pht":0.0,"ff_trk_pht":0.0},
         shed_west:{"vkt": 0.0, "cong_vkt": 0.0, "truck_vkt": 0.0,"cong_truck_vkt":0.0,
                "pht": 0.0, "ff_pht":0.0, "trk_pht":0.0,"ff_trk_pht":0.0},
         shed_north:{"vkt": 0.0, "cong_vkt": 0.0, "truck_vkt": 0.0,"cong_truck_vkt":0.0,
                "pht": 0.0, "ff_pht":0.0, "trk_pht":0.0,"ff_trk_pht":0.0},
         shed_niagara:{"vkt": 0.0, "cong_vkt": 0.0, "truck_vkt": 0.0,"cong_truck_vkt":0.0,
                "pht": 0.0, "ff_pht":0.0, "trk_pht":0.0,"ff_trk_pht":0.0}}]
cong = 0.85

truck_vols = ["@volau_htruck","@volau_mtruck","@volau_ltruck"]
auto_vols = ["@volau1_nt","@volau1_toll",\
             "@volau2_nt_nh","@volau2_nt_hov","@volau2_toll_nh","@volau2_toll_hov",\
            "@volau3_nt_nh","@volau3_nt_hov","@volau3_toll_nh","@volau3_toll_hov"]

sov_vols = ["@volau1_nt","@volau1_toll"]
hov2_vols = ["@volau2_nt_nh","@volau2_nt_hov","@volau2_toll_nh","@volau2_toll_hov"]
hov3_vols = ["@volau3_nt_nh","@volau3_nt_hov","@volau3_toll_nh","@volau3_toll_hov"]

In [4]:
for i in range(2):
    for link in ntwks[i].links():
        add = False 
        #This section determines what shed the link belongs to
        if ((link.type > 860 and link.type < 900) or (link.type >= 325 and link.type <= 328) or link.type == 330 or link.type == 332): #North
            shed = shed_north
            add = True
        elif ((link.type > 101 and link.type <200) or link.type == 100 or link.type ==333 or link.type ==329 or link.type ==331): #Central
            shed = shed_central
            add = True
        elif link.type == 101: #PD1
            shed = zones_pd1
            add = True
        elif ((link.type >199 and link.type <300) or (link.type > 899 and link.type < 950)): # East
            shed = shed_east
            add = True
        elif link.type == 820 or link.type ==850: # West
            shed = shed_west
            add = True
        elif (link.type >599 and link.type<700) or link.type ==810 or link.type == 800: #Southwest
            shed = shed_southwest
            add = True
        elif link.type == 700: # Niagara
            shed = shed_niagara
            add = True
        elif link.type >= 400 and link.type < 600: # Midwest
            shed = shed_midwest
            add = True

        if add:
             if link.volume_delay_func > 0 and link.volume_delay_func < 90 and ntwks[i].mode("I") in link.modes:
                    volume = 0.0
                    person_vol =0.0
                    truck_volume = 0.0
                    
                    for vol in sov_vols:
                        volume += link[vol]
                        person_vol += link[vol]*sov_oc
                    for vol in hov2_vols:
                        volume += link[vol]
                        person_vol += link[vol]*hov2_oc
                    for vol in hov3_vols:
                        volume += link[vol]
                        person_vol += link[vol]*hov3_oc
                        
                    for vol in truck_vols:
                        truck_volume += link[vol]

                    zones_vkt[i]["all"]["pht"] += person_vol*link.auto_time/60
                    zones_vkt[i]["all"]["trk_pht"]+= truck_volume*link.auto_time/60
                    zones_vkt[i]["all"]["ff_pht"] += person_vol*link.length/link.data2
                    zones_vkt[i]["all"]["ff_trk_pht"]+= truck_volume*link.length/link.data2

print(zones_vkt[1])
print(zones_vkt[0])

{'gs50;gs51': {'ff_pht': 0.0, 'ff_trk_pht': 0.0, 'trk_pht': 0.0, 'cong_vkt': 0.0, 'truck_vkt': 0.0, 'cong_truck_vkt': 0.0, 'vkt': 0.0, 'pht': 0.0}, 'gs10;gs11': {'ff_pht': 0.0, 'ff_trk_pht': 0.0, 'trk_pht': 0.0, 'cong_vkt': 0.0, 'truck_vkt': 0.0, 'cong_truck_vkt': 0.0, 'vkt': 0.0, 'pht': 0.0}, 'all': {'ff_pht': 155375.05778993544, 'ff_trk_pht': 49051.94524641861, 'trk_pht': 51304.62960383957, 'cong_vkt': 0.0, 'truck_vkt': 0.0, 'cong_truck_vkt': 0.0, 'vkt': 0.0, 'pht': 165000.4459541812}, 'gs20;gs21': {'ff_pht': 0.0, 'ff_trk_pht': 0.0, 'trk_pht': 0.0, 'cong_vkt': 0.0, 'truck_vkt': 0.0, 'cong_truck_vkt': 0.0, 'vkt': 0.0, 'pht': 0.0}, 'gs30;gs31': {'ff_pht': 0.0, 'ff_trk_pht': 0.0, 'trk_pht': 0.0, 'cong_vkt': 0.0, 'truck_vkt': 0.0, 'cong_truck_vkt': 0.0, 'vkt': 0.0, 'pht': 0.0}, 'gs60;gs61': {'ff_pht': 0.0, 'ff_trk_pht': 0.0, 'trk_pht': 0.0, 'cong_vkt': 0.0, 'truck_vkt': 0.0, 'cong_truck_vkt': 0.0, 'vkt': 0.0, 'pht': 0.0}, 'gf1': {'ff_pht': 0.0, 'ff_trk_pht': 0.0, 'trk_pht': 0.0, 'cong_vk

In [5]:
am_pht_lost = zones_vkt[0]["all"]["pht"] - zones_vkt[0]["all"]["ff_pht"]
md_pht_lost = zones_vkt[1]["all"]["pht"] - zones_vkt[1]["all"]["ff_pht"]

amtrk_pht_lost =  zones_vkt[0]["all"]["trk_pht"] - zones_vkt[0]["all"]["ff_trk_pht"]
mdtrk_pht_lost =  zones_vkt[1]["all"]["trk_pht"] - zones_vkt[1]["all"]["ff_trk_pht"]

In [6]:
print(am_pht_lost, md_pht_lost)
print(amtrk_pht_lost, mdtrk_pht_lost)

(144478.18144974025, 9625.388164245756)
(12166.490430390266, 2252.684357420956)
